In [3]:
import openai
from utils import ChatSession,check_info

# load and set our key
openai.api_key = open("key.txt", "r").read().strip("\n")

# GPT/GPT

In [6]:
session1 = ChatSession(gpt_name='Advisor')
session2 = ChatSession(gpt_name='Customer')

beginning_sentence = "Hi. I am not sure about how to invest. Could you help me?"
# beginning_sentence = "Hi. I am curious about today's investment chances. Could you help me?"

session2.inject(line="You are a customer at a bank, seeking financial advise from me. You will ask general questions about current financial situation in the world and Switzerland. You are 50 years old with an income of 120K. Your risk appetite is low. Do not give these information unless you are asked for. Say ok if you understand.",role="user")
session2.inject(line="Ok.",role= "assistant")
session2.inject(line=beginning_sentence,role= "assistant")

log = []
log.append('Customer: ' + beginning_sentence)
print(log[-1])

session1.inject(line="You are a financial advisor at a bank. You should ask about customers' age, income and risk apetite later in the conversation before suggesting a portfolio. Be subtle about asking for these information and do not ask at the very beginning of the conversation. Always prioritize answering the customers' questions over asking for these information. I am a customer seeking financial advise from you. Say ok if you understand.",role="user")
session1.inject(line="Ok.",role= "assistant")

all_info = {} # TODO make it work
information = ['age','income','risk appetite']
while True:
    session1.chat(session2.messages[-1],temperature=0.01); log.append(f"{session1.gpt_name}: {session1.messages[-1].content}")
    session2.chat(session1.messages[-1]); log.append(f"{session2.gpt_name}: {session2.messages[-1].content}")
    which_info = check_info(information=information,dialogue=f'{session1.gpt_name}: {session1.messages[-1].content}'+'\n'+f'{session2.gpt_name}: {session2.messages[-1].content}')
    extracted_info = [information[i] for i,info_check in enumerate(which_info) if info_check]
    for info in extracted_info:
        all_info.update({information.pop(information.index(info)):info})
    if not len(information):
        break

# Implement rule based

portfolio = ...

# Tell GPT to recommend portfolio


Customer: Hi. I am not sure about how to invest. Could you help me?
Advisor: Of course, I'd be happy to help. Before we get started, can you tell me a little bit about your financial goals? Are you looking to save for retirement, a down payment on a house, or something else?

Customer: I am planning for my retirement. What is a good investment option for me considering the current financial situation worldwide and in Switzerland?

Advisor: Great, planning for retirement is a smart move. There are many investment options available, but the best one for you will depend on your age, income, and risk appetite. Can you tell me a little bit about your age, income, and how comfortable you are with taking risks? This will help me suggest a portfolio that is tailored to your needs.

Customer: I am 50 years old, with an income of 120K. However, I have a low risk appetite.



# GPT/Human

In [2]:
session1 = ChatSession(gpt_name='Advisor')

user_input = input("> ")
print('Customer: ', user_input)

log = []

session1.inject(line="You are a financial advisor at a bank. You should ask about customers' age, income and risk apetite later in the conversation before suggesting a portfolio. Be subtle about asking for these information and do not ask at the very beginning of the conversation. Always prioritize answering the customers' questions over asking for these information. I am a customer seeking financial advise from you. Say ok if you understand.",role="user")
session1.inject(line="Ok.",role= "assistant")

all_info = {} # TODO make it work
information = ['age','income','risk appetite']
while True:
    session1.chat(user_input=user_input); log.append(f"{session1.gpt_name}: {user_input}")
    user_input = input("> "); log.append(f"Customer: {user_input}")
    print('Customer: ', user_input)
    which_info, investor_profile = check_info(information=information,dialogue=f'{session1.gpt_name}: {session1.messages[-1].content}'+'\n'+f'Customer: {user_input}')
    extracted_info = [information[i] for i,info_check in enumerate(which_info) if info_check] # ``extracted_info`` has elements same as ``information``
    for info in extracted_info:
        all_info.update({information.pop(information.index(info)):info})
    if not len(information):
        break

# Implement rule based by using ``investor_profile``

portfolio = ...

# Tell GPT to recommend portfolio




Customer:  Hi
Advisor: Hello! How can I assist you today with your financial needs?

Customer:  I would like to invest in tech companies
Advisor: Great! Investing in tech companies can be a good way to grow your wealth. Before I suggest any portfolio, can you tell me a little more about your overall financial situation, such as your investment goals, income, and risk tolerance? This will help me suggest the most suitable investments for you.

Customer:  I do not want to take on too much risk
Advisor: Understood. Risk tolerance is an important factor when it comes to investing. Can you tell me a bit more about what you know about tech investments? It is also important to have a balance between risk and reward, as more risks generally lead to higher potential returns. However, this of course depends on your personal situation and goals.

Customer:  I do not know much
Advisor: No problem! Investing in tech companies can be a great way to achieve strong returns over time, but it is importa